## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
import random
import os
import sys
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta
from subprocess import check_output
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications.vgg16 import preprocess_input
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.applications import *
import tqdm

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
import zipfile

zip_ref = zipfile.ZipFile(project_path+'train.zip', "r")
zip_ref.extractall()
zip_ref.close()

Repeat the same step for test.zip

In [0]:
import zipfile

zip_ref = zipfile.ZipFile(project_path+'test.zip', "r")
zip_ref.extractall()
zip_ref.close()

Repeat the same step for sample_submission.csv.zip

In [0]:
import zipfile

zip_ref = zipfile.ZipFile(project_path+'sample_submission.csv.zip', "r")
zip_ref.extractall()
zip_ref.close()

Repeat the same step for labels.csv.zip

In [0]:
import zipfile

zip_ref = zipfile.ZipFile(project_path+'labels.csv.zip', "r")
zip_ref.extractall()
zip_ref.close()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd
from glob import glob
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
df_labels = pd.read_csv("/content/labels.csv")
df_sample = pd.read_csv("/content/sample_submission.csv")

### Print the count of each category of Dogs given in the dataset



In [10]:
df_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [11]:
df_labels['breed'].value_counts().head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

### Get one-hot encodings of labels

In [0]:
dummies = pd.get_dummies(df_labels['breed'],columns = ['breed'])

In [13]:
dummies.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (WIDTH, HEIGHT, DEPTH)

In [0]:
x_train = []
# loop over the input images
for filepath in glob("/content/train/*.jpg"):
    img = load_img(filepath)
    arr = img_to_array(img)  # Numpy array with shape (233,233,3)
    arr = cv2.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
    #print(arr.shape) 
    x_train.append(arr)

In [0]:
x_train = np.asarray(x_train)

In [17]:
x_train.shape

(10222, 128, 128, 3)

In [18]:
x_train[0].shape

(128, 128, 3)

In [0]:
y_train = dummies

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:

x_train = np.asarray(x_train, dtype="float16")

In [0]:
x_train = x_train/255.0

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
# partition the data into training and testing splits using 70% training and 30% for validation
(Xtrain, Xval, Ytrain, Yval) = train_test_split(x_train,y_train,test_size=0.3, random_state=1)

In [23]:
print(Xtrain.shape)
print(Xval.shape)
print(Ytrain.shape)
print(Yval.shape)

(7155, 128, 128, 3)
(3067, 128, 128, 3)
(7155, 120)
(3067, 120)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [24]:
df_sample.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [0]:
test_img = df_sample['id']

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
img_rows = 128
img_cols = 128
for filepath in glob("/content/test/*.jpg"):
    img = load_img(filepath)
    arr = img_to_array(img)  # Numpy array with shape (128,128,3)
    arr = cv2.resize(arr, (img_rows,img_cols)) #Numpy array with shape (HEIGHT, WIDTH,3)
    x_test_feature.append(arr)

Normalize the test data and convert it into 4 dimensions

In [0]:
x_test = np.asarray(x_test_feature)

In [0]:
x_test = x_test/255.0

In [36]:
x_test.shape

(10357, 128, 128, 3)

In [0]:
num_classes = y_train.shape[1]

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
model = Sequential()
model.add(Conv2D(128, (5,5), padding="same", input_shape=(128, 128, 3), activation='relu'))
model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, Ytrain, validation_data=(Xval, Yval), epochs=10, batch_size=32)

Train on 7155 samples, validate on 3067 samples
Epoch 1/10
7155/7155 [==============================] - 1605s 224ms/step - loss: 4.9009 - acc: 0.0075 - val_loss: 4.7850 - val_acc: 0.0137
Epoch 2/10
7155/7155 [==============================] - 1610s 225ms/step - loss: 4.7838 - acc: 0.0103 - val_loss: 4.7832 - val_acc: 0.0137
Epoch 3/10
7155/7155 [==============================] - 1623s 227ms/step - loss: 4.7786 - acc: 0.0134 - val_loss: 4.8130 - val_acc: 0.0111
Epoch 4/10
7155/7155 [==============================] - 1636s 229ms/step - loss: 4.5560 - acc: 0.0948 - val_loss: 4.9849 - val_acc: 0.0078
Epoch 5/10
7155/7155 [==============================] - 1635s 229ms/step - loss: 2.8570 - acc: 0.4082 - val_loss: 7.2813 - val_acc: 0.0108
Epoch 6/10
5184/7155 [====================>.........] - ETA: 7:04 - loss: 1.5473 - acc: 0.6858

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
def append_ext(fn):
    return fn+".jpg"

df_labels["id"]=df_labels["id"].apply(append_ext)
df_sample["id"]=df_sample["id"].apply(append_ext)

In [0]:
from keras.applications.vgg16 import preprocess_input
image_size = 128
train_datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        rotation_range=40,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )
    
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
    


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
def get_data_generators():
   

    test_generator=test_datagen.flow_from_dataframe(
        dataframe=df_sample,
        directory="/content/drive/My Drive/Colab Notebooks/Assignments/CNN/Dog-breed-classification/test",
        x_col="id",
        y_col=None,
        batch_size=1,
        seed=42,
        shuffle=False,
        class_mode=None,
        target_size=(inputShape[0], inputShape[1]))
    
    train_generator = train_datagen.flow_from_dataframe(
        preprocessing_function=preprocess_input,
        dataframe = df_labels,
        directory='/content/drive/My Drive/Colab Notebooks/Assignments/CNN/Dog-breed-classification/train',
        x_col='id',
        y_col='breed',
        target_size=(inputShape[0], inputShape[1]),
        batch_size=128,
        class_mode='categorical',
        shuffle=True,
        seed=42,
        subset='training'
    )
    valid_generator = train_datagen.flow_from_dataframe(
        df_labels,
        directory='/content/drive/My Drive/Colab Notebooks/Assignments/CNN/Dog-breed-classification/train',
        x_col='id',
        y_col='breed',
        target_size=(inputShape[0], inputShape[1]),
        batch_size=128,
        class_mode='categorical',
        shuffle=True,
        seed=42,
        subset='validation'
    )
    
    return train_generator, valid_generator, test_generator

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [46]:
train_generator, valid_generator, test_generator = get_data_generators()

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 10357 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 0 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 10222 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


In [47]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=128,
                    validation_data=valid_generator,
                    validation_steps=128,
                    epochs=10)

128
128
Epoch 1/10


KeyboardInterrupt: ignored

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('/content/drive/My Drive/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, input_shape = [128,128,3])

Print the summary of the base_model

In [59]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
last = base_model.output

x = Flatten()(last)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
preds = Dense(120, activation='softmax')(x)
model = Model(base_model.input, preds)

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
for layer in base_model.layers:
  layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [64]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

STEP_SIZE_TRAIN=train_generator.n
STEP_SIZE_VALID=valid_generator.n

model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs=10)

Epoch 1/10


AttributeError: ignored